# Pipeline 1: Google-To-DW pipeline

The aim of this notebook is to be able to answer the questions: Is DW covering what customers want

Approach: Extract trending topics on Google and compare to what DW covers

<img src="../reports/illustrations/pipeline1.png" width=800 />

In [9]:
# Import useful libraries
import pandas as pd
import os
import sys

# Import functions from source folder
sys.path.append('../src/') 
from data.preprocess_keywords import make_cleaned_keywords_df
from data.make_datasets import get_data

# Extract trending topics from Google

In [26]:
# TODO: specify range and load it?

import pandas as pd
from pytrends.request import TrendReq as UTrendReq
import time
from pytrends.exceptions import ResponseError

'''Define header to prevent 429 errors from Google'''
GET_METHOD='get'
headers = {
    'authority': 'trends.google.com',
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-GB;q=0.9,en;q=0.8,es;q=0.7',
    'content-type': 'application/json;charset=UTF-8',
    'cookie': '__utma=10102256.1937595387.1677588086.1677588086.1678441622.2; __utmc=10102256; __utmz=10102256.1678441622.2.2.utmcsr=trends.google.com|utmccn=(referral)|utmcmd=referral|utmcct=/; __utmt=1; __utmb=10102256.13.9.1678442016068; CONSENT=YES+GB.en-GB+; HSID=AwrWd8APwv-yBWgzh; SSID=AeXCoum7ArBP5_-aa; APISID=CH4IjtEJhVzXdXGB/AFPE6uuFtOUDiSjnb; SAPISID=FcPgZF83fs0zxFml/Ad59_bwdrgg_kZ4q4; __Secure-1PAPISID=FcPgZF83fs0zxFml/Ad59_bwdrgg_kZ4q4; __Secure-3PAPISID=FcPgZF83fs0zxFml/Ad59_bwdrgg_kZ4q4; SID=TwhPHvTugfJu62Xh-HCJkOIPdoEDrL6q-6Eu9itbEI8mmKw9wzJdgT6c48lYdsNyN4E5xA.; __Secure-1PSID=TwhPHvTugfJu62Xh-HCJkOIPdoEDrL6q-6Eu9itbEI8mmKw9mFTrJ0j2r8zMRcq3v-A7Dg.; __Secure-3PSID=TwhPHvTugfJu62Xh-HCJkOIPdoEDrL6q-6Eu9itbEI8mmKw9xQIlYIR6TyZD2qXkeuopSA.; OGPC=19031986-1:; AEC=ARSKqsLpZW_sbZN2NdijlA8HPzuRHa1TPtYLHLGgaOIZpt8oJZL9PYZZYQ; SEARCH_SAMESITE=CgQI4ZcB; 1P_JAR=2023-03-10-09; NID=511=bYRTpZST7bJyL0z371h4Y79EMA1j9QqQFUpi8vJsSmiWdINx5gKruSDljEBAFfs9FYsxRrmP7vulT_MdtU2xEXQSW837vsgNY9s0i2WZAeFETmMEDrju3d_HgA2Wxy5DrFrIOaOiFu6LkpD7pY4wF4qrMZ38BzvW4NkYX_fUI7bFzHXsg24iHara1hPmPIXOSl6wQgsssfGHUntOI9PgY_eXaAEJbY7VgTr1hjNvEDlFSYOuzLvHSzo9kX9ALXA5-WOICbuLdAucZc3hJKo1dUKM51JCkzLsUHm99MWA86Icz-dmMW8ybQZhEUd2YgsBHHn5MV8uSVpcZ53n4_KL7r6sOpfWZ0ZXairmL3NH-hHz4Vyq; _gid=GA1.3.1682047475.1678441583; OTZ=6935626_48_48_123900_44_436380; _gat_gtag_UA_4401283=1; _ga=GA1.3.1937595387.1677588086; SIDCC=AFvIBn_I_znBUYDEoxfE1jUbrp_F8T607DZhlzI9o_gQoZmA4OxNjglOrH8Q8er3Cv4uzoWYkX9Z; __Secure-1PSIDCC=AFvIBn_Nhc9nywxJ_UrRYogvErcX48ygHEiBzjRRZtPe-mIwBTe_M7UbvKR4d-rAuhYyGJi-Dm0; __Secure-3PSIDCC=AFvIBn8vpeAOp5e0oAWBAETEzSClsyQlm3vQJhAQP7T7Z51q1K7zHDm_-CSGFEPasFw0sRHoJDU; _ga_VWZPXDNJJB=GS1.1.1678441583.2.1.1678442016.0.0.0',
    'origin': 'https://trends.google.com',
    'referer': 'https://trends.google.com/trends/explore?date=now%201-d&q=Adele&hl=en-GB',
    'sec-ch-ua': '"Chromium";v="110", "Not A(Brand";v="24", "Google Chrome";v="110"',
    'sec-ch-ua-arch': '"x86"',
    'sec-ch-ua-bitness': '"64"',
    'sec-ch-ua-full-version': '"110.0.5481.177"',
    'sec-ch-ua-full-version-list': '"Chromium";v="110.0.5481.177", "Not A(Brand";v="24.0.0.0", "Google Chrome";v="110.0.5481.177"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-model': '',
    'sec-ch-ua-platform': '"macOS"',
    'sec-ch-ua-platform-version': '"13.2.1"',
    'sec-ch-ua-wow64': '?0',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',
    'x-client-data': 'CIq2yQEIprbJAQjEtskBCKmdygEIkufKAQiVocsBCPyqzAEI9/XMAQib/swBCI6MzQEIlZbNAQiols0BCOGXzQEI5JfNAQjzl80BCMyYzQEI2JjNAQjzmc0BCLSazQEI0uGsAg==',
}

''' Implement header inside new TrendReq class'''
class TrendReq(UTrendReq):
    def _get_data(self, url, method=GET_METHOD, trim_chars=0, **kwargs):
        return super()._get_data(url, method=GET_METHOD, trim_chars=trim_chars, headers=headers, **kwargs)

def get_daily_trending_searches(filepath = '../data/interim/',start_date = '20190101', end_date = '20190101', geography=''):
    '''Takes a start date, end date and geography to retrieve daily Google trending searches during that date range in that geography
    also puts that dataframe inside a json file in the folder indicated by the filepath'''

    daily_trending_searches = pd.DataFrame()

    pytrend = TrendReq(                    
    # proxies=['https://34.203.233.13:80','https://35.201.123.31:880'], 
    # #hl='en-US', tz=360, timeout=(10,25)
    retries=2, backoff_factor=0.1, requests_args={'verify':False})

    for i in [dr.strftime('%Y-%m-%d') for dr in pd.date_range(start_date,end_date)]:
        try:
            pytrend.build_payload(kw_list=[''], timeframe=i + ' ' + i, geo=geography)
            df_rt_test = pytrend.related_topics()
            data = df_rt_test['']['rising']
            time.sleep(12)
            print(i)
        except ResponseError:
            print('Timeout')
        daily_trending_searches = daily_trending_searches.append(data)
    daily_trending_searches.reset_index(drop=True, inplace=True)
    daily_trending_searches['date'] = daily_trending_searches['link'].apply(lambda x: x.split('&')[1].split('date=')[1].split('+')[0])
    daily_trending_searches['location'] = geography if geography != '' else 'World'
    location = geography if geography != '' else 'World' 
    # storing the data in JSON format
    daily_trending_searches.to_json(filepath+start_date+'_'+end_date+'_'+location+'_daily_trending_searches.json', orient = 'split', compression = 'infer', index = 'true')
    
    return daily_trending_searches


# Load Google data
df_google = get_daily_trending_searches(filepath = '../data/interim/',start_date = '20190101', end_date = '20190101', geography='')
#df_google = pd.read_json(filepath+start_date+'_'+end_date+'_daily_trending_searches.json', orient ='split', compression = 'infer')

#TODO: Drop from df_google topic type = 'Topic' --> too broad and possibly repetitive vs previous trending search keyword

# Load DW data

In [27]:
# Specify wanted time range
start_date = '2019-01-01'
end_date = '2022-01-01'

In [28]:
# Clean data file in specific date range
clean_data_file = '../data/interim/clean_keywords_' + start_date + '_' + end_date + '.json'

# Generates the clean data file if it does not exist
if os.path.isfile(clean_data_file) == False:

    # Path to raw data
    data_file = '../data/raw/CMS_2010_to_June_2022_ENGLISH.json'

    # Load and extract data within time range
    df_subset = get_data(data_file, start_date, end_date)

    # Cleans keywords and saves data as a dataframe
    make_cleaned_keywords_df(df_subset, start_date, end_date)


# Loads the clean data file
df_dw = pd.read_json(clean_data_file, orient ='split', compression = 'infer')

In [29]:
## Display clean data
# cleanFocusParentCategory = Only the parent category (from src/preprocess_keywords.py function: clean_categories)
# cleanFocusCategory = all categories
# keywordStrings = raw keywords
# keywordStringsCleanAfterFuzz = Cleaned keywords (from src/preprocess_keywords.py function: make_cleaned_keywords_df)
# lastModifiedDate = raw date
# Date = just the date (in datetime format)
df_dw.head(3)

,id,lastModifiedDate,Date,keywordStrings,cleanFocusParentCategory,cleanFocusCategory,teaser,keywordStringsCleanAfterFuzz
60278,46912921,2019-01-01T03:57:28.904Z,2019-01-01,"[NASA, OSIRIS-REx, Bennu, asteroid]",Science,Science,The OSIRIS-REx spacecraft had arrived at the l...,"[nasa, osiris-rex, bennu, asteroid]"
60279,46911356,2019-01-01T06:11:50.527Z,2019-01-01,"[English Channel, migration, boats, illegal im...",Law and Justice,Law and Justice,The UK is withdrawing patrol ships from overse...,"[english channel, migration, boats, illegal im..."
60280,46909694,2019-01-01T06:14:35.563Z,2019-01-01,"[Brazil, Jair Bolsonaro, Chicago economics, Ha...",Politics,Politics,Brazil is inaugurating President Jair Bolsonar...,"[brazil, jair bolsonaro, chicago economics, ha..."


# Match Google dataset to DW timerange

In [30]:
# Extract date range in data
start_date_dw = pd.to_datetime(df_dw['Date']).min()
end_date_dw = pd.to_datetime(df_dw['Date']).max()

# Remove rows witn no category
df_dw.dropna(subset=['cleanFocusCategory'], inplace = True)
df_dw.reset_index(drop = True, inplace = True)

# Keeps only google data within DW data date range
df_google.sort_values(by ='date', inplace = True) 
mask = (pd.to_datetime(df_google['date']) > start_date_dw) & (pd.to_datetime(df_google['date']) <= end_date_dw)
df_google_subset = df_google.loc[mask].copy()

# Models: map google keywords to DW category

In [22]:
## Model 1

In [23]:
## Model 2

In [24]:
# Model comparison

# Compare trending topics and DW covered categories